In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [5]:
df = pd.read_csv("C:/Users/문도/Desktop/인지개 자료/abalone.csv", index_col = 0)
df

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
id,,,,,,,,,
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9


In [6]:
df['Sex'] = LabelEncoder().fit_transform(df['Sex'])
df

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
id,,,,,,,,,
0,2,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,2,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,2,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,1,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,0,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,2,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,2,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9


In [7]:
X = df.drop('Rings', axis = 1).values
X

array([[2.    , 0.455 , 0.365 , ..., 0.2245, 0.101 , 0.15  ],
       [2.    , 0.35  , 0.265 , ..., 0.0995, 0.0485, 0.07  ],
       [0.    , 0.53  , 0.42  , ..., 0.2565, 0.1415, 0.21  ],
       ...,
       [2.    , 0.6   , 0.475 , ..., 0.5255, 0.2875, 0.308 ],
       [0.    , 0.625 , 0.485 , ..., 0.531 , 0.261 , 0.296 ],
       [2.    , 0.71  , 0.555 , ..., 0.9455, 0.3765, 0.495 ]])

In [8]:
y = df['Rings'].values.astype(np.float32)
y

array([15.,  7.,  9., ...,  9., 10., 12.], dtype=float32)

In [9]:
scalar = StandardScaler()
X = scalar.fit_transform(X)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3341, 8), (836, 8), (3341,), (836,))

In [11]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype = torch.float32)
y_test_tensor = torch.tensor(y_test, dtype = torch.float32).view(-1, 1)

In [12]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [13]:
train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 32)

In [14]:
class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(8, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.model(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RegressionModel().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [15]:
model.train()
for epoch in range(50):
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epochs {epoch + 1}, Loss: {total_loss / len(train_loader):.4f}")
print("Training Complete")

Epochs 1, Loss: 65.0470
Epochs 2, Loss: 18.0210
Epochs 3, Loss: 7.4556
Epochs 4, Loss: 6.1116
Epochs 5, Loss: 5.7149
Epochs 6, Loss: 5.4700
Epochs 7, Loss: 5.2447
Epochs 8, Loss: 5.1508
Epochs 9, Loss: 5.0317
Epochs 10, Loss: 4.9308
Epochs 11, Loss: 4.8600
Epochs 12, Loss: 4.8064
Epochs 13, Loss: 4.7572
Epochs 14, Loss: 4.7444
Epochs 15, Loss: 4.6487
Epochs 16, Loss: 4.6138
Epochs 17, Loss: 4.5755
Epochs 18, Loss: 4.5961
Epochs 19, Loss: 4.5225
Epochs 20, Loss: 4.4998
Epochs 21, Loss: 4.5011
Epochs 22, Loss: 4.4619
Epochs 23, Loss: 4.4583
Epochs 24, Loss: 4.4103
Epochs 25, Loss: 4.3993
Epochs 26, Loss: 4.4226
Epochs 27, Loss: 4.3970
Epochs 28, Loss: 4.3897
Epochs 29, Loss: 4.3556
Epochs 30, Loss: 4.3166
Epochs 31, Loss: 4.3700
Epochs 32, Loss: 4.3095
Epochs 33, Loss: 4.2985
Epochs 34, Loss: 4.3638
Epochs 35, Loss: 4.2905
Epochs 36, Loss: 4.2885
Epochs 37, Loss: 4.2373
Epochs 38, Loss: 4.2447
Epochs 39, Loss: 4.2599
Epochs 40, Loss: 4.2813
Epochs 41, Loss: 4.2430
Epochs 42, Loss: 4.2505

In [16]:
model.eval()
preds, actuals = [], []
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        outputs = model(X_batch).cpu().numpy()
        preds.extend(outputs)
        actuals.extend(y_batch.numpy())

mse = mean_squared_error(actuals, preds)
print(f"Test MSE : {mse:.4f}")

Test MSE : 4.7887
